In [1]:
import pandas as pd
from Constraint import Problem, Constraint
from Nature import attribute_nature
import copy
from constraint_relax import search_mfs
from item_extrator import data_path, dummy_data_path
from similarity_calculator import compute_similarity

In [2]:
experiment_constraint_data = pd.read_csv('./experiment_constraint_data.csv')

In [3]:
# read experiment constraints data
problem_list = list()
unique_constraint_no = experiment_constraint_data['constraint_no'].unique()
for no in unique_constraint_no:
    problem: Problem = Problem()
    constraint_list = list()
    for index, item in experiment_constraint_data[experiment_constraint_data['constraint_no'] == no].iterrows():
        constraint = None
        if item.get('name') in ['brand', 'cpu','gpu','os']:
            constraint: Constraint = Constraint(item.get('name'), item.get('value'), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['storage', 'ram','price','weight']:
            constraint: Constraint = Constraint(item.get('name'), float(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['camera']:
            constraint: Constraint = Constraint(item.get('name'), bool(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        if constraint != None:
            constraint_list.append(constraint)
    cl_set = set(constraint_list)
    problem.constraint_list = set(constraint_list)
    problem_list.append(problem)

In [4]:
def get_df_len(df):
    if df is None:
        return 0
    return (df.shape)[0]

sol_list = []
threshold_list = [0, 0.5, 1]
for problem in problem_list:
    prob_sol_list = []
    p1 = copy.deepcopy(problem) # original constraint
    prob_sol_list.append(p1.constraint_list)
    prob_sol_list.append(get_df_len(p1.solve()))

    for threshold in threshold_list:
        p = copy.deepcopy(problem)
        mfs = search_mfs(p.constraint_list, copy.copy(p.constraint_list), dummy_data_path)
        p.relax(mfs, threshold)
        prob_sol_list.append(p.constraint_list)
        prob_items = p.solve()
        prob_sol_list.append(get_df_len(prob_items))
        if not prob_items is None:
            prob_sol_list.append((compute_similarity(p.constraint_list, prob_items))['similarity'].mean())
        else:
            prob_sol_list.append(0)
    sol_list.append(prob_sol_list)
result = pd.DataFrame(sol_list)

In [5]:
result

,0,1,2,3,4,5,6,7,8,9,10
0,"{ram: 4.0: 8: Nature.MORE, os: 11: 2: Nature.E...",4,"{ram: 4.0: 8: Nature.MORE, weight: 1.4: 9: Nat...",15,0.596817,"{price: 4118.25: 9: Nature.LESS, os: : 2: Natu...",19,0.534317,"{ram: 4.0: 8: Nature.MORE, os: : 2: Nature.EQU...",19,0.534317
1,"{price: 3399.0: 1: Nature.LESS, brand: microso...",0,"{price: 3399.0: 1: Nature.LESS, brand: microso...",0,0.000000,"{brand: microsoft|msi|hp: 9: Nature.EQUAL, ram...",0,0.000000,"{weight: : 6: Nature.LESS, storage: 256.0: 2: ...",10,0.491130
2,"{storage: 512.0: 3: Nature.MORE, price: 3618.8...",0,"{brand: huawei|acer: 9: Nature.EQUAL, price: 4...",0,0.000000,"{cpu: : 3: Nature.EQUAL, brand: huawei|acer: 9...",0,0.000000,"{price: : 4: Nature.LESS, camera: True: 5: Nat...",0,0.000000
3,"{brand: acer|asus|huawei: 8: Nature.EQUAL, pri...",0,"{price: 7501.1: 6: Nature.LESS, ram: 32.0: 8: ...",0,0.000000,"{storage: 256.0: 5: Nature.MORE, os: 10|os: 7:...",0,0.000000,"{brand: acer|asus|huawei: 8: Nature.EQUAL, ram...",0,0.000000
4,"{ram: 4.0: 2: Nature.MORE, cpu: i7|ryzen 5: 2:...",0,"{price: 5722.7: 2: Nature.LESS, cpu: i5|ryzen ...",0,0.000000,"{brand: asus: 6: Nature.EQUAL, camera: True: 4...",0,0.000000,"{storage: 256.0: 3: Nature.MORE, cpu: : 2: Nat...",4,0.687764
...,...,...,...,...,...,...,...,...,...,...,...
995,"{os: 11: 9: Nature.EQUAL, ram: 8.0: 6: Nature....",2,"{os: 11: 9: Nature.EQUAL, price: 3703.09: 5: N...",6,0.772429,"{camera: True: 8: Nature.EQUAL, price: 3703.09...",6,0.772429,"{camera: True: 8: Nature.EQUAL, price: : 5: Na...",22,0.637755
996,"{cpu: i5: 1: Nature.EQUAL, os: 10: 2: Nature.E...",0,"{os: 10: 2: Nature.EQUAL, price: 2810.49: 3: N...",0,0.000000,"{ram: 16.0: 10: Nature.MORE, price: 2810.49: 3...",0,0.000000,"{storage: 256.0: 9: Nature.MORE, price: 2810.4...",0,0.000000
997,"{camera: True: 8: Nature.EQUAL, price: 2666.48...",4,"{brand: hp|lenovo|dell|msi: 2: Nature.EQUAL, w...",8,0.762819,"{os: 10: 2: Nature.EQUAL, weight: 1.85: 9: Nat...",8,0.762819,"{storage: 512.0: 2: Nature.MORE, camera: True:...",8,0.762819
998,"{ram: 4.0: 2: Nature.MORE, price: 4436.64: 2: ...",1,"{camera: True: 4: Nature.EQUAL, ram: 4.0: 2: N...",0,0.000000,"{weight: 0.10149999999999999: 10: Nature.LESS,...",0,0.000000,"{storage: 256.0: 10: Nature.MORE, weight: : 10...",81,0.272609


In [6]:
pd.value_counts(result.iloc[:, 1] == 0)

True     571
False    429
Name: 1, dtype: int64

In [7]:
valid = ((result.iloc[:, 1] <= result.iloc[:, 3]) & (result.iloc[:, 3] <= result.iloc[:, 6]) & (result.iloc[:, 6] <= result.iloc[:, 9]))
valid_count = pd.value_counts(valid)
print(valid_count)

True     920
False     80
dtype: int64


In [8]:
valid = ((result.iloc[:, 1] < result.iloc[:, 3]) & (result.iloc[:, 3] < result.iloc[:, 6]) & (result.iloc[:, 6] < result.iloc[:, 9]))
valid_count = pd.value_counts(valid)
print(valid_count)

False    997
True       3
dtype: int64


In [9]:
result[valid]

,0,1,2,3,4,5,6,7,8,9,10
143,"{camera: True: 2: Nature.EQUAL, price: 3111.43...",12,"{weight: 1.6: 4: Nature.LESS, storage: 256: 1:...",21,0.699418,"{storage: : 1: Nature.MORE, camera: True: 2: N...",25,0.671192,"{weight: 1.6: 4: Nature.LESS, camera: True: 2:...",52,0.630681
922,"{os: 11: 1: Nature.EQUAL, brand: acer|huawei: ...",2,"{weight: 1.5: 1: Nature.LESS, brand: acer|huaw...",6,0.940682,"{price: 4211.09: 3: Nature.LESS, brand: acer|h...",7,0.906199,"{os: : 1: Nature.EQUAL, brand: acer|huawei: 10...",8,0.890682
999,"{cpu: ryzen 5: 2: Nature.EQUAL, camera: True: ...",10,"{ram: 8.0: 9: Nature.MORE, weight: 1.6615: 2: ...",11,0.436091,"{cpu: ryzen 5: 2: Nature.EQUAL, ram: 8.0: 9: N...",15,0.395601,"{ram: 8.0: 9: Nature.MORE, storage: 256.0: 9: ...",52,0.401316
